In [1]:
import pandas as pd
from inference import inf_model

In [2]:
import numpy as np

def kl_divergence_normal(mu1, sigma1, mu2, sigma2):
    # Compute KL divergence between two normal distributions
    return np.log(sigma2 / sigma1) + (sigma1**2 + (mu1 - mu2)**2) / (2 * sigma2**2) - 0.5

In [163]:
df = pd.read_csv('data/two_treatment/dt5_with_context.csv' ,index_col=0)


In [164]:
# Policy and Inference Model
inference_model =  inf_model.NormalKnownVariance(
    
    prior_mean=0, prior_variance=1, variance=1
)
methods =["mean_fill (global)", "mean_fill (individual)", "tr_mean_fill (global)", "tr_mean_fill (individual)", "KNN_fill", "cluster_fill",
               "DR_fill"]

number_of_actions = 2
mean_p0 = 2
mean_p1 = 0
var_p0 = 1
var_p1 = 1

In [165]:
mean_a, var_a = inference_model.update_posterior(df, number_of_actions)
mean_a0 = mean_a[0] 
mean_a1 = mean_a[1]

var_a0 = var_a[0] 
var_a1 = var_a[1]

In [176]:
block_length = 5
t = 25

In [177]:
dis_res = pd.read_pickle(f'result_output/block{block_length}/result_block{block_length}wct{t}_raw.pkl')
dis_res

mean_fill (global)                                              \
    mean of treatment 0 mean of treatment 1 variance of treatment 0   
0              2.016014            0.020236                0.000479   
1              2.014682            0.020849                0.000479   
2              2.012035            0.021078                0.000479   
3              2.016003            0.025837                0.000480   
4              2.009095            0.021078                0.000479   
..                  ...                 ...                     ...   
995            2.011546            0.024414                0.000479   
996            2.010759            0.022439                0.000479   
997            2.014216            0.023344                0.000479   
998            2.011437            0.020149                0.000479   
999            2.016472            0.021524                0.000479   

                            mean_fill (individual)                      \
    variance of treatment 1    mean of treatment 0 mean of treatment 1   
0                  0.001773               2.012306            0.020485   
1                  0.001776               2.011362            0.020162   
2                  0.001779               2.008206            0.020272   
3                  0.001770               2.008365            0.022333   
4                  0.001779               2.007575            0.020354   
..                      ...                    ...                 ...   
995                0.001773               2.009822            0.020030   
996                0.001776               2.008796            0.018232   
997                0.001773               2.010482            0.020272   
998                0.001776               2.005551            0.019925   
999                0.001776               2.010804            0.020180   

                                                    tr_mean_fill (global)  \
    variance of treatment 0 variance of treatment 1   mean of treatment 0   
0                  0.000443                0.001761              2.019562   
1                  0.000443                0.001764              2.023316   
2                  0.000443                0.001757              2.022582   
3                  0.000443                0.001761              2.021389   
4                  0.000442                0.001767              2.017146   
..                      ...                     ...                   ...   
995                0.000443                0.001764              2.017397   
996                0.000443                0.001761              2.019511   
997                0.000443                0.001761              2.019870   
998                0.000443                0.001764              2.019837   
999                0.000443                0.001764              2.020920   

                         ...                KNN_fill                          \
    mean of treatment 1  ... variance of treatment 0 variance of treatment 1   
0              0.020326  ...                0.000443                0.001764   
1              0.019718  ...                0.000442                0.001767   
2              0.018423  ...                0.000442                0.001767   
3              0.022094  ...                0.000443                0.001764   
4              0.019144  ...                0.000443                0.001761   
..                  ...  ...                     ...                     ...   
995            0.017952  ...                0.000442                0.001767   
996            0.019907  ...                0.000443                0.001761   
997            0.019783  ...                0.000443                0.001761   
998            0.016859  ...                0.000442                0.001767   
999            0.017987  ...                0.000443                0.001761   

           cluster_fill                                              \
    

In [178]:
kl_div_p0 = []
kl_div1_p0 = []
kl_div_a0 = []

kl_div_p1 = []
kl_div1_p1 = []
kl_div_a1 = []

for method in methods:
    mean_out = dis_res[method]
    
    mean_0 = mean_out['mean of treatment 0'].mean()
    mean_1 = mean_out['mean of treatment 1'].mean()
    
    var_0 = mean_out['variance of treatment 0'].mean()
    var_1 = mean_out['variance of treatment 1'].mean()
    
    kl_div_p0.append(kl_divergence_normal(mean_p0,np.sqrt(var_p0), mean_0,np.sqrt(var_0)))
    kl_div1_p0.append(kl_divergence_normal(mean_p0,np.sqrt(var_p0), mean_0,np.sqrt(1)))
    kl_div_a0.append(kl_divergence_normal(mean_a0,np.sqrt(var_a0), mean_0,np.sqrt(var_0)))
    
    kl_div_p1.append(kl_divergence_normal(mean_p1,np.sqrt(var_p1), mean_1,np.sqrt(var_1)))
    kl_div1_p1.append(kl_divergence_normal(mean_p1,np.sqrt(var_p1), mean_1,np.sqrt(1)))
    kl_div_a1.append(kl_divergence_normal(mean_a1,np.sqrt(var_a1), mean_1,np.sqrt(var_1)))

In [179]:
kl_result = {
    'method': ["mean_fill (global)", "mean_fill (individual)", "tr_mean_fill (global)", "tr_mean_fill (individual)", "KNN_fill", "cluster_fill",
               "DR_fill"],
    'dictance from posterior dis for tr 0': kl_div_a0,
    'dictance from popultion dis for tr 0': kl_div_p0,
    'dictance from popultion dis with var = 1 for tr 0': kl_div1_p0,
    
    'dictance from posterior dis for tr 1': kl_div_a1,
    'dictance from popultion dis for tr 1': kl_div_p1,
    'dictance from popultion dis with var = 1 for tr 1': kl_div1_p1
    
      
}
kl_result = pd.DataFrame(kl_result)
kl_result

,method,dictance from posterior dis for tr 0,dictance from popultion dis for tr 0,dictance from popultion dis with var = 1 for tr 0,dictance from posterior dis for tr 1,dictance from popultion dis for tr 1,dictance from popultion dis with var = 1 for tr 1
0,mean_fill (global),0.039221,1039.135787,0.000114,0.010357,278.237409,0.000222
1,mean_fill (individual),0.001337,1125.075211,0.000043,0.008132,280.107377,0.000207
2,tr_mean_fill (global),0.064213,1039.201533,0.000143,0.003108,278.201850,0.000168
3,tr_mean_fill (individual),0.082981,1125.366706,0.000161,0.007853,280.080461,0.000206
4,KNN_fill,0.064380,1125.287139,0.000143,0.015179,280.144167,0.000249
5,cluster_fill,0.782143,1125.213765,0.000142,0.005096,280.179573,0.000185
6,DR_fill,0.064922,1125.284865,0.000143,0.004899,280.110441,0.000184


In [180]:
kl_result.to_pickle(f'result_output/kl_block{block_length}wct{t}_raw.pkl')